<a href="https://colab.research.google.com/github/leivakuro765/learm-machine-learning/blob/main/nlp/2_SVD_and_NMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##### 

Singular Value Decomposition (SVD) and Non-negative Matrix Factorization (NMF)

### RECURSO
[ truncated SVD LSI details](https://scikit-learn.org/stable/modules/decomposition.html#lsa)

[The 20 newsgroups text dataset¶](https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset)


only cumpute the **k** largest singular values where **K** is a user-specified parameter.

𝑿 ≈ 𝑿ₖ = UₖEₖVₖVᵀ  
UₖEₖ

the use SVD is that in the effects of synonymy and polysemy (both of wihch roughly meab there are multiple meanings per word )

In [ ]:
import pprint

import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from sklearn.datasets import fetch_20newsgroups
from sklearn import decomposition

# config

%matplotlib inline
np.set_printoptions(suppress=True)


the dataset to be used will be [fetch_20newsgroups](https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset)

The data is organized into **20 different newsgroups**, each corresponding to a different topic. Some of the newsgroups are closely related to each other (eg comp.sys.ibm.pc.hardware / comp.sys.mac.hardware), while others are completely unrelated (eg misc .forsale/soc.religion.christian). Here's a list of the 20 newsgroups, divided (more or less) by topic:

In [ ]:
catg = ['alt.atheism', 
        'talk.religion.misc',
        'comp.graphics',
        'sci.space']
train_new =fetch_20newsgroups(subset='train', categories=catg) 
test_new =fetch_20newsgroups(subset='test', categories=catg) 

list(train_new.target_names)

In [ ]:
list(test_new.target_names)


In [ ]:
(train_new.filenames.shape, train_new.target.shape,
test_new.filenames.shape, test_new.target.shape)


In [ ]:
print('\n'.join(train_new.data[5:7]))

[_stop_words](https://stackoverflow.com/questions/68620436/cannot-import-name-stop-words-from-sklearn-feature-extraction)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from sklearn.feature_extraction import _stop_words
(list(_stop_words.ENGLISH_STOP_WORDS))[:20]

## Stemming and Lemmatization

For grammatical reasons of a document, different ways of representing a text will be used.

the goal of lemmalization is to reduce the inflectional and sometimes derivative forms of a common base-form pala

The most common algorithm for deriving English, and one that has repeatedly been shown to be very effective empirically, is [Porter's] Algorithm(https://tartarus.org/martin/PorterStemmer/). Not only do you use Porter's algorithm, there are others but they do not give a good result like porter's despite being a bit simpler in this [document](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1. html) two more are shown

In [ ]:
from nltk import stem

def lem(word):
  """
  parm:
    word_list:: list is word list 
  return:
    tuple the two size 
    0) lemalizado with WordNet
    2) lemalizado with algoritmo Porter

  """
  wnl = stem.WordNetLemmatizer()
  porter = stem.PorterStemmer()
  rest = (
      [wnl.lemmatize(word) for word in word], 
      [porter.stem(word) for word in word]
      ) 

  return rest

foot =  ['feet','foot', 'foots','footing']

print('WordNet: ',lem(foot)[0])
print('Porter:  ',lem(foot)[1])



[spacy](https://spacy.io/)

In [ ]:
!pip install -U spacy
# You will then need to download the English model:
!python -m spacy download en_core_web_sm

import spacy
# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")
doc = nlp('feet foot foots footing')

[token.lemma_ for token in doc]

In [ ]:
# size stop word is 326
(sorted(list(nlp.Defaults.stop_words))[:20],
len(nlp.Defaults.stop_words) )
 

In [ ]:
# size stop word is 179
(sorted(nltk.corpus.stopwords.words('english'))[:20]
,len(nltk.corpus.stopwords.words('english')))

In [ ]:
# size stop word is 318
(sorted(list(_stop_words.ENGLISH_STOP_WORDS))[:20]
,len(_stop_words.ENGLISH_STOP_WORDS))

## Data Processing

After playing for a while with the modules that we will use, it is time to process the data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(train_new.data).todense()
vectors.shape

In [ ]:
print(len(train_new.data), vectors.shape)
vocab = np.array(vectorizer.get_feature_names())
vocab.shape

In [ ]:
vocab[200:260]

In [ ]:
%time U, s, Vh = linalg.svd(vectors, full_matrices=False)

In [ ]:
U.shape, s.shape, Vh.shape
plt.plot(s)

In [ ]:
plt.plot(s[:20])

In [ ]:
num_top_words=8
def show_topics(a):
    global num_top_words
    top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_top_words-1:-1]]
    topic_words = ([top_words(t) for t in a])
    return [' '.join(t) for t in topic_words]

In [ ]:
show_topics(Vh[20:30])

In [ ]:
m, n = vectors.shape
d = 5

# model NMF
clf = decomposition.NMF(n_components=d, random_state=1)
W1 = clf.fit_transform(vectors)
H1 = clf.components_

show_topics(H1)

In [ ]:
vectorizer_tfidf = TfidfVectorizer(stop_words='english')
vectors_tfidf = vectorizer_tfidf.fit_transform(train_new.data) # (documents, 

In [ ]:
train_new.data[:1]

In [ ]:
W1 = clf.fit_transform(vectors_tfidf)
H1 = clf.components_
show_topics(H1)

In [ ]:
plt.plot(clf.components_[0])
clf.reconstruction_err_